In [39]:
VIDEO_FRAMERATE = 30 #nr of frames per second
#it takes my pc 3 min 15 s with framerate 0.5. For 25 fps, estimated is 2 h and 45 mins
PATH_TO_VIDEO = '2022-06-14_16-38-43_S04_eye-tracking-video-world_frame.mp4'
PATH_TO_INFO_AND_CROSS = "2022-06-14_16-38-43_streamLog_actionNet-wearables_S04.hdf5"
VIDEO_ID = "S04_01"
FRAMES_SAVE_PATH = "actionNet/" + VIDEO_ID
CALIBRATION_SHIFT = 14*60 + 21 # the first part of the video is usless
PIKLE_PATH = "actionNet/records.pkl"

VIDEO_SAMPLING_RATE = 1/VIDEO_FRAMERATE
import h5py, numpy as np

## Video transformation into frames

In [40]:
import os
os.makedirs(FRAMES_SAVE_PATH, exist_ok=True)

In [41]:
import cv2
def transform_to_frames(frameRate, path_to_video, frames_save_path, shift=0): 
    vidcap = cv2.VideoCapture(path_to_video)
    framecount = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    video_fps = vidcap.get(cv2.CAP_PROP_FPS)
    tot_num_secs = framecount/video_fps - shift
    completion = 0
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            cv2.imwrite(frames_save_path+"/img_"+f'{count:010d}'+".jpg", image)     # save frame as JPG file
        return hasFrames
    sec = 0
    count=0
    success = getFrame(sec)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec+shift)
        if round(sec/tot_num_secs, 2) > completion:
            completion = round(sec/tot_num_secs, 2)
            print(str(completion*100) + "%")

In [42]:
video_fps = cv2.VideoCapture(PATH_TO_VIDEO).get(cv2.CAP_PROP_FPS)
video_fps

29.669551601030665

In [43]:
transform_to_frames(VIDEO_SAMPLING_RATE, PATH_TO_VIDEO, FRAMES_SAVE_PATH,CALIBRATION_SHIFT)

1.0%
2.0%
3.0%
4.0%
5.0%
6.0%
7.000000000000001%
8.0%
9.0%
10.0%
11.0%
12.0%
13.0%
14.000000000000002%
15.0%
16.0%
17.0%
18.0%
19.0%
20.0%
21.0%
22.0%
23.0%
24.0%
25.0%
26.0%
27.0%
28.000000000000004%
28.999999999999996%
30.0%
31.0%
32.0%
33.0%
34.0%
35.0%
36.0%
37.0%
38.0%
39.0%
40.0%
41.0%
42.0%
43.0%
44.0%
45.0%
46.0%
47.0%
48.0%
49.0%
50.0%
51.0%
52.0%
53.0%
54.0%
55.00000000000001%
56.00000000000001%
56.99999999999999%
57.99999999999999%
59.0%
60.0%
61.0%
62.0%
63.0%
64.0%
65.0%
66.0%
67.0%
68.0%
69.0%
70.0%
71.0%
72.0%
73.0%
74.0%
75.0%
76.0%
77.0%
78.0%
79.0%
80.0%
81.0%
82.0%
83.0%
84.0%
85.0%
86.0%
87.0%
88.0%
89.0%
90.0%
91.0%
92.0%
93.0%
94.0%
95.0%
96.0%
97.0%
98.0%
99.0%
100.0%


## Labels and dataframe (records)

In [50]:



h5_file = h5py.File(PATH_TO_INFO_AND_CROSS)

device_name = 'experiment-activities'
stream_name = 'activities'

# Get the timestamped label data.
# As described in the HDF5 metadata, each row has entries for ['Activity', 'Start/Stop', 'Valid', 'Notes'].
activity_datas = h5_file[device_name][stream_name]['data']
activity_times_s = h5_file[device_name][stream_name]['time_s']
activity_times_s = np.squeeze(np.array(activity_times_s))  # squeeze (optional) converts from a list of single-element lists to a 1D list
# Convert to strings for convenience.
activity_datas = [[x.decode('utf-8') for x in datas] for datas in activity_datas]

# Combine start/stop rows to single activity entries with start/stop times.
#   Each row is either the start or stop of the label.
#   The notes and ratings fields are the same for the start/stop rows of the label, so only need to check one.
exclude_bad_labels = True # some activities may have been marked as 'Bad' or 'Maybe' by the experimenter; submitted notes with the activity typically give more information
activities_labels = []
activities_start_times_s = []
activities_end_times_s = []
activities_ratings = []
activities_notes = []
for (row_index, time_s) in enumerate(activity_times_s):
  label    = activity_datas[row_index][0]
  is_start = activity_datas[row_index][1] == 'Start'
  is_stop  = activity_datas[row_index][1] == 'Stop'
  rating   = activity_datas[row_index][2]
  notes    = activity_datas[row_index][3]
  if exclude_bad_labels and rating in ['Bad', 'Maybe']:
    continue
  # Record the start of a new activity.
  if is_start:
    activities_labels.append(label)
    activities_start_times_s.append(time_s)
    activities_ratings.append(rating)
    activities_notes.append(notes)
  # Record the end of the previous activity.
  if is_stop:
    activities_end_times_s.append(time_s)

In [51]:
import pandas as pd
action_net = pd.read_pickle("./action-net/ActionNet_train.pkl")
action_dict = {}
for i,row in action_net[["description", "labels"]].iterrows():
    desc = row[0]
    label = row[1]
    action_dict[desc] = label

In [52]:
labels_dict = {}
i=0
for label in set(action_net["labels"]):
    labels_dict[label] = i
    i+=1

{'Clean',
 'Clear',
 'Get/Put',
 'Load',
 'Open/Close',
 'Peel',
 'Pour',
 'Set',
 'Slice',
 'Spread',
 'Stack',
 'Unload'}

In [53]:
#TODO decide if we want to merge the same actions into one
#TODO reduce to only actions
from datetime import datetime
import pandas as pd

#seeding uuid 
import uuid
import random
rd = random.Random()
rd.seed(9341)

#"start_timestamp", "stop_timestamp"
records = []
shift = datetime.utcfromtimestamp(min(activities_start_times_s))
for i, label in enumerate(activities_labels) :
    #activities_end_times_s
    #start_timestamp = datetime.timestamp(datetime.utcfromtimestamp(activities_start_times_s[i]) - shift)
    #stop_timestamp = datetime.timestamp(datetime.utcfromtimestamp(activities_end_times_s[i]) - shift)
    start_frame = int((datetime.utcfromtimestamp(activities_start_times_s[i]) - shift).total_seconds() * VIDEO_FRAMERATE)
    stop_frame = int((datetime.utcfromtimestamp(activities_end_times_s[i]) - shift).total_seconds() * VIDEO_FRAMERATE)
    narration = label
    verb = action_dict[label]
    verb_class = labels_dict[verb]
    uuid_str = uuid.UUID(int=rd.getrandbits(128))
    records.append([uuid_str, VIDEO_ID, start_frame,stop_frame, narration, verb, verb_class])

records_pd = pd.DataFrame(records,columns=["uuid", "video_id","start_frame", "stop_frame", "narration", "verb", "verb_class"])

In [54]:
records_pd

,uuid,video_id,start_frame,stop_frame,narration,verb,verb_class
0,d4f63e5e-f85c-fc3f-02dc-c80aca18b35b,S04_01,0,2287,Get/replace items from refrigerator/cabinets/d...,Get/Put,9
1,c6f5c839-708e-c4cd-8b48-95cd7a4af83f,S04_01,2709,4157,Peel a cucumber,Peel,6
2,c7908761-b083-8833-13f4-16386b9d1262,S04_01,4682,5779,Peel a cucumber,Peel,6
3,829b6cb8-2528-d8bc-1f21-803cea2c3d63,S04_01,6108,7494,Peel a cucumber,Peel,6
4,e6fdddf6-dd65-2e46-cbb5-6972bfdcadf2,S04_01,7926,9183,Clear cutting board,Clear,2
5,50412c8a-9113-8423-ad35-dbd12fabac19,S04_01,9500,10956,Slice a cucumber,Slice,1
6,ac0db17b-3e92-dcbf-2ae1-800616851c99,S04_01,11488,12867,Slice a cucumber,Slice,1
7,a43e5e1c-f092-ea92-5f95-5aea0176e195,S04_01,13228,14384,Slice a cucumber,Slice,1
8,a6543ec7-188c-8f05-bfa8-51806d0ccf9b,S04_01,14943,16293,Clear cutting board,Clear,2
9,4c652a9d-9ca8-f23d-57c8-8a91ee463283,S04_01,16595,18701,Get/replace items from refrigerator/cabinets/d...,Get/Put,9


In [55]:
records_pd.to_pickle(PIKLE_PATH)